# Imports

In [2]:
import pandas as pd

Initialize functions

In [17]:
def categorize(df, categories):
    for category in categories:
        df[category] = pd.Categorical(df[category]).codes
    return df

def numberize(df, numerics):
    for numeric in numerics:
        df[numeric] = df[numeric].astype("float")
    return df

# Function to clean year_built values
def clean_year_built(year):
    try:
        # Attempt to convert the value to an integer
        year = int(float(year))  # Handles both integer and float strings
        year_str = str(year)
        
        # Ensure the string is exactly 4 characters long
        if len(year_str) > 4:
            return int(year_str[:4])
        elif len(year_str) < 4:
            return int(year_str.ljust(4, '0'))  # Pads with zeros if less than 4 digits
        else:
            return year
    except (ValueError, TypeError):
        # Return NaN for non-numeric values or None
        return pd.NA

load and prepare the data

In [29]:
df = pd.read_csv("flatfox.csv")
df.drop(df[df["offer_type"] != "RENT"].index, inplace=True)
# remove columns with wrong year
df.drop(df[df["year_built"] < 1800].index, inplace=True)
df.drop(df[df["year_built"] > 2024].index, inplace=True)
# NaN  in floors is replaced by 0
# year_built has unrealistic values (for example 19 Million)
#df['year_built'] = df['year_built'].apply(clean_year_built)

df.fillna({"year_built": df["year_built"].mean(), "floor": 0.0, "year_renovated": df["year_built"], "number_of_rooms": 0.0, "livingspace": 0.0}, inplace=True)


df.dropna(subset=["price_display"], inplace=True)
df.drop([
    "pk",
    "slug",
    "url",
    "short_url",
    "moving_date",
    "rent_net",
    "rent_charges",
    "rent_gross",
    "offer_type",
    "submit_url",
    "status",
    "created",
    "reference",
    "ref_property",
    "ref_house",
    "ref_object",
    "alternative_reference",
    "published",
    "short_title",
    "public_title",
    "pitch_title",
    "description_title",
    "description",
    "attributes",
    "public_address",
    "video_url",
    "tour_url",
    "website_url",
    "live_viewing_url",
    "cover_image",
    "images",
    "documents",
    "agency",
    "rent_title",
    "surface_property",
    "surface_living",
    "surface_usable",
    "surface_usable_minimum",
    "volume",
    "space_display",
    "street"], axis=1, inplace=True)

categories = [
    "object_category",
    "object_type",
    "price_display_type",
    "price_unit",
    "city",
    "moving_date_type"
]
numerics = [
    "is_furnished",
    "is_temporary",
    "is_selling_furniture",
    "reserved"
]
categorize(df, categories)
numberize(df, numerics)
df.describe()
#df.head()
#df.isna().sum()
#df.dtypes

C:\Users\lirla\AppData\Local\Temp\ipykernel_9196\2080806206.py:1: DtypeWarning: Columns (52) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("flatfox.csv")


,object_category,object_type,price_display,price_display_type,price_unit,number_of_rooms,floor,is_furnished,is_temporary,is_selling_furniture,zipcode,city,latitude,longitude,year_built,year_renovated,moving_date_type,reserved,livingspace
count,20672.000000,20672.000000,20672.000000,20672.000000,20672.000000,20672.000000,20672.000000,20672.000000,20672.000000,20672.000000,20672.000000,20672.000000,20672.000000,20672.000000,20672.000000,20672.000000,20672.000000,20672.000000,20672.000000
mean,2.306115,19.428986,1581.677873,0.954334,1.137190,1.928285,1.299971,0.143334,0.065402,0.023849,5562.717637,1119.948336,47.158640,8.023957,1984.806247,1978.824804,1.053793,0.000339,80.139754
std,2.742482,18.110377,1535.049146,0.208764,0.627099,1.839069,2.110613,0.350422,0.247241,0.152581,2744.774139,683.722513,0.404462,0.848056,21.628201,164.519116,0.802859,0.018399,671.466531
min,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,-8.000000,0.000000,0.000000,0.000000,1000.000000,0.000000,45.826182,5.991881,1800.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,400.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3097.000000,500.000000,46.956114,7.466299,1984.863253,1984.863253,0.000000,0.000000,0.000000
50%,0.000000,16.000000,1460.000000,1.000000,1.000000,1.500000,1.000000,0.000000,0.000000,0.000000,5430.000000,1100.000000,47.346152,8.203656,1984.863253,1984.863253,1.000000,0.000000,58.000000
75%,5.000000,37.000000,2142.750000,1.000000,1.000000,3.500000,2.000000,0.000000,0.000000,0.000000,8152.000000,1732.000000,47.430642,8.587501,1984.863253,2008.000000,2.000000,0.000000,94.000000
max,8.000000,58.000000,62539.000000,1.000000,4.000000,10.500000,31.000000,1.000000,1.000000,1.000000,9657.000000,2157.000000,47.793652,10.364311,2024.000000,2027.000000,2.000000,1.000000,90000.000000


count all years: 21040 

count 1500-2024: 20725

count 1800-2024: 20672 


Write clean data to csv

In [21]:
df.to_csv("flatfox_clean.csv", index=False)